### 从零开始实现线性回归模型

In [82]:
import torch
import random

#### 生成数据

In [91]:
def get_data(w, b, num_examples):
    X = torch.normal(0,1, (num_examples, len(w)))
    y = torch.matmul(X,w)+ b #两个矩阵之间的乘法(1000,2) * (1,2)
    y += torch.normal(0,0.01, y.shape)
    return X, y.reshape((-1,1))

true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = get_data(true_w, true_b, 1000)
print(features[0],'\n', labels[0])

tensor([0.1516, 0.6281]) 
 tensor([2.3601])


#### 读取数据集

In [92]:
#定义一个小批量的iter数据集读取器来获取部分数据
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices) #打乱标签
    
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]
    
batch_size = 10
for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[-0.2204, -0.0670],
        [ 0.2235,  1.5720],
        [-0.6561, -0.8191],
        [-0.8057,  0.6768],
        [ 1.4756,  0.2708],
        [ 0.8840, -0.6264],
        [-0.6116,  0.2095],
        [-0.0490,  1.8906],
        [ 0.8388, -0.0773],
        [ 0.0875, -0.0455]]) 
 tensor([[ 3.9812],
        [-0.6827],
        [ 5.6876],
        [ 0.2835],
        [ 6.2316],
        [ 8.1123],
        [ 2.2607],
        [-2.3209],
        [ 6.1560],
        [ 4.5228]])


### 初始化参数 

In [85]:
w = torch.normal(0,0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

### 定义模型 

In [86]:
def linreg(X, w, b):
    #线性回归模型
    return torch.matmul(X,w)+b

### 定义损失函数 

In [93]:
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

### 定义优化算法（梯度下降法） 

In [94]:
def sgd(params, lr, batch_size):
    with torch.no_grad(): #梯度下降算法
        for param in params:
            param -= lr * param.grad/batch_size
            param.grad.zero_()

### 训练 

In [98]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        # 因为l形状是(batch_size,1)，而不是一个标量。l中的所有元素被加到一起，
        # 并以此计算关于[w,b]的梯度
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch{epoch + 1}, loss{float(train_l.mean()):f}')

epoch1, loss0.000052
epoch2, loss0.000052
epoch3, loss0.000052


In [99]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

w的估计误差: tensor([ 0.0003, -0.0003], grad_fn=<SubBackward0>)
b的估计误差: tensor([-0.0005], grad_fn=<RsubBackward1>)


## 简洁实现

In [110]:
def get_data(num_examples, w, b):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X,w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1,1))

true_w = torch.tensor([2, -2.4])
true_b = 5
features, labels = get_data(1000, true_w, true_b)